In [79]:
#download raw data

import pandas as pd
import yfinance as yf

# Download S&P 500 data from yfinance since 1971
df = yf.download('^GSPC', start='1971-08-15', end='2023-09-22')

# Add a column for daily change
df['Change'] = df['Adj Close'].pct_change(periods=1)

# Remove other columns except daily change
df.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close'], inplace=True)

# Remove 1st row (due to no data for Column 'Çhange')
df = df.iloc[1:]

# Reset Index
df = df.reset_index()

df.info

[*********************100%%**********************]  1 of 1 completed


<bound method DataFrame.info of             Date    Change
0     1971-08-17  0.012454
1     1971-08-18 -0.013901
2     1971-08-19 -0.004462
3     1971-08-20  0.001732
4     1971-08-23  0.009356
...          ...       ...
13134 2023-09-15 -0.012160
13135 2023-09-18  0.000721
13136 2023-09-19 -0.002151
13137 2023-09-20 -0.009395
13138 2023-09-21 -0.016401

[13139 rows x 2 columns]>

In [80]:
#prepare df_train

for i in range(0, 13139, 50):
    x = i + 1
    #print(x)
    start_date = df.iloc[i:x]['Date'].tolist()
    #print(ts)
    extract_date = start_date[0].strftime('%Y-%m-%d')
    #print(extract_date)
    df[extract_date] = df['Change'].shift(-i)
    
df.drop(columns=['Date', 'Change'], inplace=True)
df_train = df[0:200]

df_train = df_train.drop(['2022-12-20', '2023-03-06', '2023-05-16', '2023-07-28'], axis=1)

df_train = df_train.transpose()

C:\Users\szech\AppData\Local\Temp\ipykernel_12828\380791017.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[extract_date] = df['Change'].shift(-i)
C:\Users\szech\AppData\Local\Temp\ipykernel_12828\380791017.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[extract_date] = df['Change'].shift(-i)
C:\Users\szech\AppData\Local\Temp\ipykernel_12828\380791017.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider j

In [87]:
df_train.info

<bound method DataFrame.info of                  0         1         2         3         4         5    \
1971-08-17  0.012454 -0.013901 -0.004462  0.001732  0.009356  0.011587   
1971-10-27 -0.010027  0.001813  0.002874 -0.015176  0.004095  0.018566   
1972-01-07 -0.000386 -0.001450  0.003194 -0.000579 -0.005792  0.003884   
1972-03-20 -0.003058 -0.008365  0.001406  0.008517 -0.002135 -0.002046   
1972-05-31 -0.007431  0.001461  0.000365 -0.008293 -0.005606 -0.005175   
...              ...       ...       ...       ...       ...       ...   
2021-12-22  0.010180  0.006224  0.013839 -0.001010  0.001402 -0.002990   
2022-03-07 -0.029518 -0.007234  0.025698 -0.004292 -0.012962 -0.007421   
2022-05-17  0.020170 -0.040395 -0.005834  0.000146  0.018555 -0.008121   
2022-07-29  0.014208 -0.002823 -0.006662  0.015638 -0.000777 -0.001626   
2022-10-10 -0.007492 -0.006519 -0.003291  0.025966 -0.023663  0.026480   

                 6         7         8         9    ...       190       191  \


In [83]:
#prepare df_test

import pandas as pd
import yfinance as yf

# Download S&P 500 data from yfinance since 1971
df_test = yf.download('^GSPC', start='2022-12-8', end='2023-09-28')

# Add a column for daily change
df_test['Change'] = df_test['Adj Close'].pct_change(periods=1)

# Remove other columns except daily change
df_test.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close'], inplace=True)

# Remove 1st row (no data)
df_test = df_test.iloc[1:]

df_test = df_test.rename(columns={'Change': '2022-12-8'})

df_test.head(10)

df_test = df_test.transpose()

[*********************100%%**********************]  1 of 1 completed


In [84]:
df_test.head(5)

Date,2022-12-09,2022-12-12,2022-12-13,2022-12-14,2022-12-15,2022-12-16,2022-12-19,2022-12-20,2022-12-21,2022-12-22,...,2023-09-14,2023-09-15,2023-09-18,2023-09-19,2023-09-20,2023-09-21,2023-09-22,2023-09-25,2023-09-26,2023-09-27
2022-12-8,-0.00735,0.014279,0.00729,-0.006053,-0.024922,-0.011138,-0.009008,0.001037,0.014868,-0.014452,...,0.00843,-0.01216,0.000721,-0.002151,-0.009395,-0.016401,-0.002296,0.004023,-0.014735,0.000229


In [85]:
#find top 10 similarity score from 259 time series

import pandas as pd
import numpy as np

# Load historical time series from CSV
historical_data = df_train

# Load latest time series from CSV
latest_data = df_test

# Calculate similarity scores
similarity_scores = []
for i in range(len(historical_data)):
    historical_series = historical_data.iloc[i].values
    similarity_score = np.corrcoef(historical_series, latest_data.values.flatten())[0, 1]
    similarity_scores.append(similarity_score)

# Find indices of top 10 most similar time series
top_10_indices = np.argsort(similarity_scores)[-10:]

# Get the top 10 most similar time series
top_10_similar_series = historical_data.iloc[top_10_indices]

# Print the top 10 most similar time series
print(top_10_similar_series)

                 0         1         2         3         4         5    \
2002-04-26 -0.013889 -0.010099  0.010765  0.008859 -0.001749 -0.010262   
1985-01-30  0.001172  0.001338 -0.005567  0.009629  0.001442 -0.000997   
2009-11-10 -0.000064  0.005032 -0.010259  0.005739  0.014468  0.000919   
1999-09-22  0.002241 -0.022968 -0.002382  0.004658 -0.000865 -0.010786   
1980-07-15 -0.005916  0.002766  0.015130  0.004941  0.003851 -0.002612   
2010-04-07 -0.005877  0.003374  0.006684  0.001767  0.000685  0.011150   
1992-05-27  0.001847  0.011088 -0.003335  0.004695 -0.009106  0.002636   
1975-03-12 -0.009128  0.001794  0.012181  0.014748 -0.010231 -0.009280   
1976-05-18  0.001682 -0.000790  0.008104 -0.007255 -0.017974  0.000503   
2009-04-08  0.011783  0.038053  0.002533 -0.020064  0.012549  0.015539   

                 6         7         8         9    ...       190       191  \
2002-04-26 -0.019340 -0.003021  0.037504 -0.014547  ...  0.013050  0.006779   
1985-01-30  0.007704  0.002

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.012454,-0.013901,-0.004462,0.001732,0.009356,0.011587,0.000100,-0.001693,0.002394,-0.009554,...,-0.001872,0.002156,0.009823,0.009635,0.006515,0.000820,0.004828,0.001360,0.001811,-0.002801
1,-0.010027,0.001813,0.002874,-0.015176,0.004095,0.018566,-0.001264,-0.003481,-0.000741,0.000742,...,-0.002325,0.000932,0.000093,0.009405,0.008210,0.007777,0.002633,0.001630,0.000723,0.001536
2,-0.000386,-0.001450,0.003194,-0.000579,-0.005792,0.003884,0.002998,0.003375,-0.001634,0.000000,...,0.006704,0.002554,0.000819,-0.004455,-0.008219,-0.006262,-0.010656,0.006837,0.006419,-0.001294
3,-0.003058,-0.008365,0.001406,0.008517,-0.002135,-0.002046,-0.001212,-0.006345,0.006667,0.002612,...,-0.004790,-0.003352,-0.007244,0.006255,0.004058,0.005417,0.009578,0.008894,0.003946,-0.001422
4,-0.007431,0.001461,0.000365,-0.008293,-0.005606,-0.005175,-0.003437,-0.003915,0.001404,0.005046,...,0.012602,0.003067,-0.001922,-0.003852,0.000615,0.005445,0.004368,-0.007480,-0.005082,-0.012066
